##__Lo que nos falta del arbol__
1.  __Conjunto de datos categorico__ El ejemplo fue diseñado para datos de entrada con atributos de entrada numéricos u ordinales, experimente con datos de entrada categóricos y divisiones que pueden usar la igualdad en lugar de la clasificación.
2. __Regresión__ . Adapte el árbol para la regresión utilizando una función de costo y un método diferentes para crear nodos terminales.

3. __Graficar y visualizar el arbol__. Buscar cual libreria nos puede ayudar a visualizar el arbol 


In [ ]:
from random import seed
from random import randrange
from csv import reader
#import pandas as pd

#model
class Tree :
	# Calculate the Gini index for a split dataset
	def gini_index(groups, classes):
		# count all samples at split point
		n_instances = float(sum([len(group) for group in groups]))
		# sum weighted Gini index for each group
		gini = 0.0
		for group in groups:
			size = float(len(group))
			# avoid divide by zero
			if size == 0:
				continue
			score = 0.0
			# score the group based on the score for each class
			for class_val in classes:
				p = [row[-1] for row in group].count(class_val) / size
				score += p * p
			# weight the group score by its relative size
			gini += (1.0 - score) * (size / n_instances)
		return gini
		
	# Load a CSV file
	def load_csv(filename):
		file = open(filename, "rt")
		lines = reader(file)
		dataset = list(lines)
		return dataset
 
class Classifier :
	values = {}
	split_number = 0
	# Convert string column to float
	def str_column_to_float(dataset, column):
		for row in dataset:
			row[column] = float(row[column].strip())

	# Split a dataset into k folds
	def cross_validation_split(dataset, n_folds):
		dataset_split = list()
		dataset_copy = list(dataset)
		fold_size = int(len(dataset) / n_folds)
		for i in range(n_folds):
			fold = list()
			while len(fold) < fold_size:
				index = randrange(len(dataset_copy))
				fold.append(dataset_copy.pop(index))
			dataset_split.append(fold)
		return dataset_split
	
	# Calculate accuracy percentage
	def accuracy_metric(actual, predicted):
		correct = 0
		for i in range(len(actual)):
			if actual[i] == predicted[i]:
				correct += 1
		return correct / float(len(actual)) * 100.0
	
	# Evaluate an algorithm using a cross validation split
	def evaluate_algorithm(dataset, algorithm, n_folds, *args):
		folds = Classifier.cross_validation_split(dataset, n_folds)
		scores = list()
		for fold in folds:
			train_set = list(folds)
			train_set.remove(fold)
			train_set = sum(train_set, [])
			test_set = list()
			for row in fold:
				row_copy = list(row)
				test_set.append(row_copy)
				row_copy[-1] = None
			predicted = algorithm(train_set, test_set, *args)
			actual = [row[-1] for row in fold]
			accuracy = Classifier.accuracy_metric(actual, predicted)
			scores.append(accuracy)
		return scores
	
	# Split a dataset based on an attribute and an attribute value
	def test_split(index, value, dataset):
		left, right = list(), list()
		for row in dataset:
			if row[index] < value:
				left.append(row)
			else:
				right.append(row)
		return left, right
	

	#Metodo a modificar
	# Select the best split point for a dataset
	def get_split(dataset):
		class_values = list(set(row[-1] for row in dataset))
		b_index, b_value, b_score, b_groups = 999, 999, 999, None
		for index in range(len(dataset[0])-1):
			for row in dataset:
				groups = Classifier.test_split(index, row[index], dataset)
				gini = Tree.gini_index(groups, class_values)
				#print(row[index])
				if gini < b_score:
					b_index, b_value, b_score, b_groups = index, row[index], gini, groups
		#print('Splitter', str(b_value), 'Ginni', str(b_score))
		Classifier.split_number = Classifier.split_number + 1 
		Classifier.values.setdefault(Classifier.split_number,b_score)
		return {'index':b_index, 'value':b_value, 'groups':b_groups}
	
	# Create a terminal node value
	def to_terminal(group):
		outcomes = [row[-1] for row in group]
		return max(set(outcomes), key=outcomes.count)
	
	# Create child splits for a node or make terminal
	def split(node, max_depth, min_size, depth):
		left, right = node['groups']
		del(node['groups'])
		# check for a no split
		if not left or not right:
			node['left'] = node['right'] = Classifier.to_terminal(left + right)
			return
		# check for max depth
		if depth >= max_depth:
			node['left'], node['right'] = Classifier.to_terminal(left), Classifier.to_terminal(right)
			return
		# process left child
		if len(left) <= min_size:
			node['left'] = Classifier.to_terminal(left)
		else:
			node['left'] = Classifier.get_split(left)
			Classifier.split(node['left'], max_depth, min_size, depth+1)
		# process right child
		if len(right) <= min_size:
			node['right'] = Classifier.to_terminal(right)
		else:
			node['right'] = Classifier.get_split(right)
			Classifier.split(node['right'], max_depth, min_size, depth+1)
	
	# Build a decision tree
	def build_tree(train, max_depth, min_size):
		print('Root:')
		root = Classifier.get_split(train)
		Classifier.split(root, max_depth, min_size, 1)
		print(root)
		return root
	
	# Make a prediction with a decision tree
	def predict(node, row):
		if row[node['index']] < node['value']:
			if isinstance(node['left'], dict):
				return Classifier.predict(node['left'], row)
			else:
				return node['left']
		else:
			if isinstance(node['right'], dict):
				return Classifier.predict(node['right'], row)
			else:
				return node['right']
	
	# Classification and Regression Tree Algorithm
	def decision_tree(train, test, max_depth, min_size):
		tree = Classifier.build_tree(train, max_depth, min_size)
		predictions = list()
		for row in test:
			prediction = Classifier.predict(tree, row)
			predictions.append(prediction)
		return (predictions)

In [ ]:
class Main():
  train_set = {}
  test_set = {}
  def decisionTree(n_folds ,max_depth, min_size, filename):
   isTested = False
   isTrained = False
   dataset = Tree.load_csv(filename)
   seed(1)
   train, test = Main.train_test_split(dataset)
   while((isTrained and isTested)==False):
     if(isTrained == False):
       print("Entrenamiento del arbol: ")
   # convert string attributes to integers
       for i in range(len(dataset[0])):
	       Classifier.str_column_to_float(train, i)
   # evaluate algorithm
       scores = Classifier.evaluate_algorithm(train, Classifier.decision_tree, n_folds, max_depth, min_size)
       print('Scores: %s' % scores)
       print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
       train_set = Classifier.values
       print(train_set)
       print("\n")
       Classifier.values = {}
       Classifier.split_number = 0
       isTrained = True
     else:
       print("Resultados de test: ")
       dataset = Tree.load_csv(filename)
       for i in range(len(dataset[0])):
	       Classifier.str_column_to_float(test, i)
       scores = Classifier.evaluate_algorithm(test, Classifier.decision_tree, n_folds, max_depth, min_size)
       print('Scores: %s' % scores)
       print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
       test_set = Classifier.values
       print(test_set)
       print("\n")
       isTested = True

  def train_test_split(dataset, split=0.60):
	  train = list()
	  train_size = split * len(dataset)
	  dataset_copy = list(dataset)
	  while len(train) < train_size:
		  index = randrange(len(dataset_copy))
		  train.append(dataset_copy.pop(index))
	  return train, dataset_copy    


In [ ]:
1
n_folds =int(input("n_folds: ")) 
max_depth =int( input("max_depth: "))
min_size =int(input("min_size: "))
filename = input("Name for your file: ")
Main.decisionTree(n_folds,max_depth, min_size, filename)
#n_folds: Pliegues
#max_depth: profundidad maxima con la que trabajara el arbol 
#min_size: numero minimo de filas
#imprime la presicion de clasificacion se imprime la precisión de clasificación promedio en cada pliegue (Scores), 
#así como el rendimiento promedio en todos los pliegues(Mean Accurracy).

n_folds: 3
max_depth: 3
min_size: 3
Name for your file: data_banknote_authentication (1).txt
Entrenamiento del arbol: 
Root:
{'index': 0, 'value': 0.75736, 'left': {'index': 1, 'value': 5.1815, 'left': {'index': 0, 'value': 0.3292, 'left': 1.0, 'right': 1.0}, 'right': {'index': 0, 'value': -2.6989, 'left': 1.0, 'right': 0.0}}, 'right': {'index': 2, 'value': -4.5852, 'left': {'index': 0, 'value': 5.7456, 'left': 1.0, 'right': 0.0}, 'right': {'index': 0, 'value': 1.6032, 'left': 0.0, 'right': 0.0}}}
Root:
{'index': 0, 'value': 0.3292, 'left': {'index': 1, 'value': 8.1881, 'left': {'index': 0, 'value': -0.39816, 'left': 1.0, 'right': 1.0}, 'right': {'index': 0, 'value': -2.7419, 'left': 1.0, 'right': 0.0}}, 'right': {'index': 2, 'value': -4.2931, 'left': {'index': 0, 'value': 4.2899, 'left': 1.0, 'right': 0.0}, 'right': {'index': 0, 'value': 0.85574, 'left': 0.0, 'right': 0.0}}}
Root:
{'index': 0, 'value': 0.3292, 'left': {'index': 1, 'value': 8.1881, 'left': {'index': 0, 'value': -0.3603

In [ ]:

from random import seed
from random import randrange
from csv import reader
 
# Split a dataset into a train and test set
def train_test_split(dataset, split=0.60):
	train = list()
	train_size = split * len(dataset)
	dataset_copy = list(dataset)
	while len(train) < train_size:
		index = randrange(len(dataset_copy))
		train.append(dataset_copy.pop(index))
	return train, dataset_copy

def load_csv(filename):
		file = open(filename, "rt")
		lines = reader(file)
		dataset = list(lines)
		return dataset
 
# test train/test split
seed(1)
filemane = "test.csv"
#dataset = [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]]
dataset = load_csv(filename);
train, test = train_test_split(dataset)
print(train)
print(test)



[['959', '1', 'Moore, Mr. Clarence Bloomfield', 'male', '47', '0', '0', '113796', '42.4', '', 'S'], ['1183', '3', 'Daly, Miss. Margaret Marcella Maggie""', 'female', '30', '0', '0', '382650', '6.95', '', 'Q'], ['1303', '1', 'Minahan, Mrs. William Edward (Lillian E Thorpe)', 'female', '37', '1', '0', '19928', '90', 'C78', 'Q'], ['1284', '3', 'Abbott, Master. Eugene Joseph', 'male', '13', '0', '2', 'C.A. 2673', '20.25', '', 'S'], ['923', '2', 'Jefferys, Mr. Clifford Thomas', 'male', '24', '2', '0', 'C.A. 31029', '31.5', '', 'S'], ['1023', '1', 'Gracie, Col. Archibald IV', 'male', '53', '0', '0', '113780', '28.5', 'C51', 'C'], ['952', '3', 'Dika, Mr. Mirko', 'male', '17', '0', '0', '349232', '7.8958', '', 'S'], ['1148', '3', 'Mahon, Mr. John', 'male', '', '0', '0', 'AQ/4 3130', '7.75', '', 'Q'], ['1287', '1', 'Smith, Mrs. Lucien Philip (Mary Eloise Hughes)', 'female', '18', '1', '0', '13695', '60', 'C31', 'S'], ['1125', '3', 'Linehan, Mr. Michael', 'male', '', '0', '0', '330971', '7.8792'

In [ ]:
import pandas as pd
from csv import reader
def load_csv(filename):
	file = open(filename)
	lines = reader(file)
	dataset = list(lines)
	return dataset

dataset= pd.read_csv("data_banknote_authentication.txt")
dataset



,3.6216,8.6661,-2.8073,-0.44699,0
0,4.54590,8.16740,-2.4586,-1.46210,0
1,3.86600,-2.63830,1.9242,0.10645,0
2,3.45660,9.52280,-4.0112,-3.59440,0
3,0.32924,-4.45520,4.5718,-0.98880,0
4,4.36840,9.67180,-3.9606,-3.16250,0
...,...,...,...,...,...
1366,0.40614,1.34920,-1.4501,-0.55949,1
1367,-1.38870,-4.87730,6.4774,0.34179,1
1368,-3.75030,-13.45860,17.5932,-2.77710,1
1369,-3.56370,-8.38270,12.3930,-1.28230,1


In [ ]:
dataset.columns

Index(['3.6216', '8.6661', '-2.8073', '-0.44699', '0'], dtype='object')

In [ ]:
x=3.14
y=23
z = "Hola mundo"

print(type(x))
print(type(y))
print(type(z))

<class 'float'>
<class 'int'>
<class 'str'>


In [ ]:
#Metodo a modificar
# Select the best split point for a dataset
def get_split(dataset):
  class_values = list(set(row[-1] for row in dataset))
  b_index, b_value, b_score, b_groups = 999, 999, 999, None
  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups = Classifier.test_split(index, row[index], dataset)
      gini = Tree.gini_index(groups, class_values)
      print(row[index]+' '+gini)
      if gini < b_score:
        b_index, b_value, b_score, b_groups = index, row[index], gini, groups
  return {'index':b_index, 'value':b_value, 'groups':b_groups}